## 

## Import libraries

In [14]:
import pandas as pd

## Import data

In [15]:
data = pd.read_json('./data/proposicoes_join_temas.json')
data.head()

,id,uri,siglaTipo,codTipo,numero,ano,ementa,tema
0,1198010,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2234,2023,Altera redação de dispositivos do artigo 4° da...,Defesa e Segurança
1,2345485,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,1,2023,Institui a Política Nacional de Longo Prazo.,Administração Pública
2,2345498,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2,2023,"Altera a Lei nº 12.529, de 30 de novembro de 2...",Economia
3,2345498,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2,2023,"Altera a Lei nº 12.529, de 30 de novembro de 2...",Finanças Públicas e Orçamento
4,2345625,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,3,2023,Cria o Protocolo Não é Não de atendimento à mu...,Direito Penal e Processual Penal


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103902 entries, 0 to 103901
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         103902 non-null  int64 
 1   uri        103902 non-null  object
 2   siglaTipo  103902 non-null  object
 3   codTipo    103902 non-null  int64 
 4   numero     103902 non-null  int64 
 5   ano        103902 non-null  int64 
 6   ementa     103902 non-null  object
 7   tema       98133 non-null   object
dtypes: int64(4), object(4)
memory usage: 6.3+ MB


## EDA

Laws proposals

In [5]:
unique_laws_proposal = len(data['id'].unique())

print(f'{unique_laws_proposal} Unique laws')

71746 Unique laws


Show all the PL's unique themes

In [6]:
temas = data['tema'].unique().tolist()

print( len(temas), "unique themes\n", temas )

32 unique themes
 ['Defesa e Segurança', 'Administração Pública', 'Economia', 'Finanças Públicas e Orçamento', 'Direito Penal e Processual Penal', 'Direitos Humanos e Minorias', 'Indústria, Comércio e Serviços', 'Comunicações', 'Meio Ambiente e Desenvolvimento Sustentável', 'Saúde', 'Esporte e Lazer', 'Homenagens e Datas Comemorativas', 'Educação', 'Direito Constitucional', None, 'Relações Internacionais e Comércio Exterior', 'Viação, Transporte e Mobilidade', 'Trabalho e Emprego', 'Previdência e Assistência Social', 'Direito Civil e Processual Civil', 'Direito e Justiça', 'Agricultura, Pecuária, Pesca e Extrativismo', 'Turismo', 'Cidades e Desenvolvimento Urbano', 'Política, Partidos e Eleições', 'Direito e Defesa do Consumidor', 'Energia, Recursos Hídricos e Minerais', 'Estrutura Fundiária', 'Arte, Cultura e Religião', 'Ciência, Tecnologia e Inovação', 'Processo Legislativo e Atuação Parlamentar', 'Ciências Sociais e Humanas']


One-Hot-Encoding the themes

In [7]:
data["count"] = 1

data = data.drop(
    columns=[
        "uri",
        "siglaTipo",
        "codTipo",
        "numero",
        "ano"
    ]
)

# pivot table
data_pivot = data.pivot_table(
    index=["id", "ementa"],
    columns="tema",
    values="count",
    aggfunc="sum",
    fill_value=0,
).reset_index()

The number of records is reduced because, apareantly, there are some PL's that have no themes associated.

The no-themes PL's are removed from the dataset on the pivot table operation.

In [8]:
print(data_pivot.shape)

(65977, 33)


Count the frequency of each theme

In [9]:
# Calculate the percentage of each theme
data_frequency = data_pivot.drop(columns=['id','ementa']).sum().sort_values(ascending=False)
# to pandas dataframe
data_frequency = pd.DataFrame(data_frequency).reset_index()
data_frequency.columns = ['tema', 'frequencia']
data_frequency['percentual'] = 100*data_frequency['frequencia'] / len( data_pivot )
data_frequency.head(32)

,tema,frequencia,percentual
0,Direitos Humanos e Minorias,9103,13.797232
1,Trabalho e Emprego,8326,12.619549
2,Saúde,8108,12.289131
3,Finanças Públicas e Orçamento,7606,11.528260
4,Administração Pública,7293,11.053852
5,Direito Penal e Processual Penal,6102,9.248678
6,Educação,5560,8.427179
7,"Indústria, Comércio e Serviços",4834,7.326796
8,"Viação, Transporte e Mobilidade",4756,7.208573
9,Defesa e Segurança,3781,5.730785


Showing a few examples of the PL's ementas

In [32]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    print(data.groupby("tema").apply(lambda df: df.head(2))[["ementa"]])

                                                                                              ementa
tema                                                                                                
Administração Pública                       1           Institui a Política Nacional de Longo Prazo.
                                            44     Dispõe sobre o exercício de funções de magisté...
Agricultura, Pecuária, Pesca e Extrativismo 85     Cria o Roteiro Turístico dos Cafés da Alta Mog...
                                            168    Altera os artigos 429 e 430 da Consolidação da...
Arte, Cultura e Religião                    141    Reconhece a Festa de San Gennaro, realizada no...
                                            162    Institui a meia entrada para doadores regulare...
Cidades e Desenvolvimento Urbano            89     Altera a Lei nº 8.036, de 11 de maio de 1990, ...
                                            110    Altera a Lei nº 14.312, de 14 de março d

## Preprocessing

Join themes with less than 1% relative frequency as 'Other'

In [10]:
join_themes = [
    "Ciência, Tecnologia e Inovação",
    "Direito e Justiça",
    "Relações Internacionais e Comércio Exterior",
    "Turismo",
    "Processo Legislativo e Atuação Parlamentar",
    "Direito Constitucional",
    "Ciências Sociais e Humanas",
]

# 1 if the law has at least one of the themes above, 0 otherwise
data_pivot["Outro"] = (
    data_pivot[join_themes]
    .sum(axis=1)
    .apply(lambda x: 0 if x == 0 else 1)
)

# Remove the themes that were joined together
data_themes_removed = data_pivot.drop(columns=join_themes)

## Save data 

In [13]:
data_themes_removed.to_parquet('./data/proposicoes_temas_one_hot_encoding.parquet')